0. Load libraries / Init Spark

In [ ]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col, expr, udf, rank
from pyspark.sql.types import IntegerType

In [47]:
spark = SparkSession.builder.appName("spark-exercises").getOrCreate()
sc = spark.sparkContext

1. Read csvs as pyspark dataframes, you can find them in the server (departments, dept_emp,dept_manager,employees,salaries,titles)  
‘’movies.csv” will not be used yet.

In [ ]:
df_departments = spark.read.csv(
    "csv/departments.csv", header=True, inferSchema=True
)
df_dept_emp = spark.read.csv(
    "csv/dept_emp.csv", header=True, inferSchema=True
)
df_dept_manager = spark.read.csv(
    "csv/dept_manager.csv", header=True, inferSchema=True
)
df_employees = spark.read.csv(
    "csv/employees.csv", header=True, inferSchema=True
)
df_salaries = spark.read.csv(
    "csv/salaries.csv", header=True, inferSchema=True
)
df_titles = spark.read.csv(
    "csv/titles.csv", header=True, inferSchema=True
)

2. Print cell on the first row , first column of the “departments’” spark-dataframe you've created

In [88]:
# print
df_departments.first()[0]

'd009'

In [89]:
# verify
df_departments.head(5)

[Row(dept_no='d009', dept_name='Customer Service'),
 Row(dept_no='d005', dept_name='Development'),
 Row(dept_no='d002', dept_name='Finance'),
 Row(dept_no='d003', dept_name='Human Resources'),
 Row(dept_no='d001', dept_name='Marketing')]

3. On the employees dataframe transform gender column as following:

In [92]:
# Made sure it won't break if ran more than once
df_employees = df_employees.withColumn(
    "gender",
    expr(
        """
    CASE 
    WHEN gender in ('M', 'Male') THEN 'Male'
    WHEN gender in ('F', 'Female') THEN 'Female'
    END
    """
    ),
)

In [93]:
# verify
df_employees.show(5)

+------+----------+----------+---------+------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|
+------+----------+----------+---------+------+----------+
| 10001|1953-09-02|    Georgi|  Facello|  Male|1986-06-26|
| 10002|1964-06-02|   Bezalel|   Simmel|Female|1985-11-21|
| 10003|1959-12-03|     Parto|  Bamford|  Male|1986-08-28|
| 10004|1954-05-01| Chirstian|  Koblick|  Male|1986-12-01|
| 10005|1955-01-21|   Kyoichi| Maliniak|  Male|1989-09-12|
+------+----------+----------+---------+------+----------+
only showing top 5 rows



4. How many rows exist in the salaries table?

In [94]:
df_salaries.count()

2844047

5. What is the datatype in each column of the salaries table?

In [96]:
df_salaries.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- from_date: date (nullable = true)
 |-- to_date: date (nullable = true)



6. Most of our tables contain dates. Make sure that their datatype is date, otherwise change it to date. (not timestamp)

In [97]:
df_salaries.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- from_date: date (nullable = true)
 |-- to_date: date (nullable = true)



In [98]:
df_departments.printSchema()

root
 |-- dept_no: string (nullable = true)
 |-- dept_name: string (nullable = true)



In [99]:
df_dept_emp.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- dept_no: string (nullable = true)
 |-- from_date: date (nullable = true)
 |-- to_date: date (nullable = true)



In [100]:
df_dept_manager.printSchema()

root
 |-- dept_no: string (nullable = true)
 |-- emp_no: integer (nullable = true)
 |-- from_date: date (nullable = true)
 |-- to_date: date (nullable = true)



In [101]:
df_employees.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- hire_date: date (nullable = true)



In [102]:
df_titles.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- from_date: date (nullable = true)
 |-- to_date: date (nullable = true)



In [103]:
# They are all dates

7. Create a simple function to calculate age from date of birth. (current year-year of birth). Use this custom function to create a new column on employees dataframe with each person's age.

In [117]:
# define the function
def get_age(date: datetime.date) -> int:
    return (datetime.now().date() - date).days // 365

In [118]:
# create the udf
my_udf = udf(get_age, IntegerType())

In [ ]:
df_employees = df_employees.withColumn(
    "age", my_udf(df_employees["birth_date"])
)

In [120]:
df_employees.head(5)

[Row(emp_no=10001, birth_date=datetime.date(1953, 9, 2), first_name='Georgi', last_name='Facello', gender='Male', hire_date=datetime.date(1986, 6, 26), age=71),
 Row(emp_no=10002, birth_date=datetime.date(1964, 6, 2), first_name='Bezalel', last_name='Simmel', gender='Female', hire_date=datetime.date(1985, 11, 21), age=61),
 Row(emp_no=10003, birth_date=datetime.date(1959, 12, 3), first_name='Parto', last_name='Bamford', gender='Male', hire_date=datetime.date(1986, 8, 28), age=65),
 Row(emp_no=10004, birth_date=datetime.date(1954, 5, 1), first_name='Chirstian', last_name='Koblick', gender='Male', hire_date=datetime.date(1986, 12, 1), age=71),
 Row(emp_no=10005, birth_date=datetime.date(1955, 1, 21), first_name='Kyoichi', last_name='Maliniak', gender='Male', hire_date=datetime.date(1989, 9, 12), age=70)]

8. How many employees does the company currently have?

In [ ]:
# They are distinct, but just making sure
df_employees.select("emp_no").distinct().count()

300024

9. How many employees does the company currently have?

In [ ]:
# check schema
df_dept_emp.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- dept_no: string (nullable = true)
 |-- from_date: date (nullable = true)
 |-- to_date: date (nullable = true)



In [ ]:
df_dept_emp.groupby("dept_no").count().orderBy("dept_no").show()

+-------+-----+
|dept_no|count|
+-------+-----+
|   d001|20211|
|   d002|17346|
|   d003|17786|
|   d004|73485|
|   d005|85707|
|   d006|20117|
|   d007|52245|
|   d008|21126|
|   d009|23580|
+-------+-----+



10. Produce a table with two columns. department names & number of employees

In [ ]:
# We will have to create two tables and join them
df_departments.createOrReplaceTempView("departments")
df_dept_emp.createOrReplaceTempView("dept_emp")

In [ ]:
# check schema
df_departments.printSchema()

root
 |-- dept_no: string (nullable = true)
 |-- dept_name: string (nullable = true)



In [143]:
spark.sql(
    """
    select d.dept_name, count(de.emp_no) as employe_count
    from departments d
    join dept_emp de using (dept_no)
    group by d.dept_name
    order by d.dept_name
"""
).show()

+------------------+-------------+
|         dept_name|employe_count|
+------------------+-------------+
|  Customer Service|        23580|
|       Development|        85707|
|           Finance|        17346|
|   Human Resources|        17786|
|         Marketing|        20211|
|        Production|        73485|
|Quality Management|        20117|
|          Research|        21126|
|             Sales|        52245|
+------------------+-------------+



11. What is the highest paid job title on average? How many employees does the company occupy with this title?

In [ ]:
# I will need to join titles and salaries tables
df_salaries = df_salaries.createOrReplaceTempView("salaries")
df_titles = df_titles.createOrReplaceTempView("titles")

In [151]:
# Highest paid job
spark.sql(
    """
    select t.title, cast(avg(s.salary) as int) avg_salary
    from titles t
    join salaries s using (emp_no)
    group by t.title
    order by avg_salary desc
    limit 1
"""
).show()

+------------+----------+
|       title|avg_salary|
+------------+----------+
|Senior Staff|     70470|
+------------+----------+



12. Create a master view that gathers the following info for all current employees:  
    1. employee id
    2. employee salary
    3. name of department
    4. manager's id
    5. manager's name
    6. Managere's full name
    7. employee's full name
    8. current title
    9. date hired in company
    10. date started current role
    11. years in company hint:  use function created for age this time instead of current year - birth year pass hire year

In [ ]:
# I pretty much have to join everything... I am missing only the dept_manager table I think
df_dept_manager.createOrReplaceTempView("dept_manager")

In [205]:
# register udf for use with sparkSQL
spark.udf.register("get_age", get_age, IntegerType())

<function __main__.get_age(date: <method 'date' of 'datetime.datetime' objects>) -> int>

In [232]:
# join into the master
df_master = spark.sql(
    """
    select e.emp_no, s.salary, d.dept_name, dm.emp_no as manager_no, e2.first_name manager_name, concat(e2.first_name, ' ', e2.last_name) manager_full_name, concat(e.first_name, ' ', e.   last_name) employee_full_name, t.title, e.hire_date, t.from_date, get_age(e.hire_date) years_in_company
    --select *
    from employees e
    join salaries s on s.emp_no = e.emp_no and s.to_date = '9999-01-01'
    join dept_emp de on de.emp_no = e.emp_no and de.to_date = '9999-01-01'
    join departments d using (dept_no)
    join dept_manager dm on dm.dept_no = d.dept_no and dm.to_date = '9999-01-01'
    join employees e2 on dm.emp_no = e2.emp_no
    join titles t on e.emp_no = t.emp_no and t.to_date = '9999-01-01'
    order by e.emp_no
"""
)

In [233]:
# create master view
df_master.createOrReplaceTempView("master_view")

In [234]:
# 13, hired after ... and changed titles
spark.sql(
    """
    select *
    from master_view
    where hire_date > '1998-12-31' and hire_date <> from_date
    """
).limit(5).show()

+------+------+-----------+----------+------------+-----------------+------------------+------------------+----------+----------+----------------+
|emp_no|salary|  dept_name|manager_no|manager_name|manager_full_name|employee_full_name|             title| hire_date| from_date|years_in_company|
+------+------+-----------+----------+------------+-----------------+------------------+------------------+----------+----------+----------------+
| 10298| 43923| Production|    110420|       Oscar|   Oscar Ghazalie|  Dietrich DuCasse|             Staff|1999-03-30|1999-04-27|              26|
| 11697| 45619|Development|    110567|        Leon|    Leon DasSarma|      JoAnne Merey|  Technique Leader|1999-11-06|1999-12-10|              25|
| 11754| 47936|   Research|    111534|      Hilary|    Hilary Kambil|    Malu Magliocco|             Staff|1999-08-23|1999-11-06|              25|
| 11829| 48098| Production|    110420|       Oscar|   Oscar Ghazalie|  Chaosheng Pettis|Assistant Engineer|1999-02-17|

In [235]:
# 14. rank salaries dept-based (sql)
spark.sql(
    """
    select *, 
        rank() over(partition by  dept_name order by salary desc) dept_rank
    from master_view
    """
).sample(.00003).show()

+------+------+---------+----------+------------+-----------------+------------------+---------------+----------+----------+----------------+---------+
|emp_no|salary|dept_name|manager_no|manager_name|manager_full_name|employee_full_name|          title| hire_date| from_date|years_in_company|dept_rank|
+------+------+---------+----------+------------+-----------------+------------------+---------------+----------+----------+----------------+---------+
|216227|101040|    Sales|    111133|       Hauke|      Hauke Zhang|   Rosalie Provine|   Senior Staff|1992-10-31|2000-10-31|              32|     9366|
|499005| 81891|    Sales|    111133|       Hauke|      Hauke Zhang|Juichirou Fujisawa|          Staff|1998-02-09|1998-02-09|              27|    24216|
|295045| 68513| Research|    111534|      Hilary|    Hilary Kambil|    Arvind Holburn|Senior Engineer|1993-01-23|1993-01-23|              32|     6969|
+------+------+---------+----------+------------+-----------------+------------------+--

In [236]:
# 14b. Using DF
window_func = Window.partitionBy('dept_name').orderBy(col('Salary').desc())
df_master_with_rank = df_master.withColumn('dept_rank', rank().over(window_func))
df_master_with_rank.sample(.00003).show()

+------+------+----------------+----------+------------+-----------------+-------------------+------------+----------+----------+----------------+---------+
|emp_no|salary|       dept_name|manager_no|manager_name|manager_full_name| employee_full_name|       title| hire_date| from_date|years_in_company|dept_rank|
+------+------+----------------+----------+------------+-----------------+-------------------+------------+----------+----------+----------------+---------+
| 36903|100598|           Sales|    111133|       Hauke|      Hauke Zhang|       Shan Vickson|Senior Staff|1990-01-19|1990-01-19|              35|     9661|
|100961| 67325|Customer Service|    111939|     Yuchang|  Yuchang Weedman|    Marke Penttonen|Senior Staff|1987-03-09|1993-03-08|              38|     7798|
| 29405| 65972|        Research|    111534|      Hilary|    Hilary Kambil|Poornachandra Viele|       Staff|1995-09-04|1995-09-04|              29|     8013|
| 18660| 46471|     Development|    110567|        Leon|  

In [237]:
# 15. Print all views
spark.catalog.listTables()

[Table(name='departments', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='dept_emp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='dept_manager', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='employees', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='master_view', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='salaries', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='titles', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [ ]:
# 16. write to parquet
df_master_with_rank.write.parquet('parquet/')